# Notebook to create aggregated reference files (json) into monthly or yearly zarrs

In [1]:
import sys

In [2]:
sys.path.insert(0,'/home/jovyan/filesystem_spec')

In [3]:
from fsspec_reference_maker.combine import MultiZarrToZarr
import fsspec
import xarray as xr
import os, glob
# xr.set_options(display_style='text')

In [4]:
fsspec.__file__

'/home/jovyan/filesystem_spec/fsspec/__init__.py'

In [5]:
gateway_cluster = False
from dask.distributed import Client, LocalCluster
from dask_gateway import Gateway

# Dask gateway
if gateway_cluster:
    gateway = Gateway()
    clusters = gateway.list_clusters()
    if not clusters:
        print('Creating new cluster. Please wait for this to finish.')
        cluster = gateway.new_cluster()
    else:
        print(f'An existing cluster was found. Connected to cluster \033[1m{clusters[0].name}\033[0m')
        cluster=gateway.connect(clusters[0].name)
    cluster.adapt(minimum=1, maximum=30)
    client = cluster.get_client()
    client.wait_for_workers(n_workers=1)

# Local cluster
else:
    cluster = LocalCluster()
    client = Client(cluster)

display(cluster)
display(client)

/env/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46153 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:38737 Dashboard: http://127.0.0.1:46153/status,Cluster Workers: 8 Cores: 32 Memory: 30.06 GB


In [6]:
# Check a file
def open_dataset(fn,preprocess=None):
    so = dict(
        target_protocol='s3',
        target_options=dict(profile='default'),
        remote_protocol='s3',
        remote_options=dict(profile='default'), 
        fo=fn
    )
    ds = xr.open_dataset('reference://',
                         engine='zarr',
                         storage_options=so,
                         chunks={})
    
    if preprocess is not None:
        ds = preprocess(ds)
    
    return ds

fn = 's3://imos-data-pixeldrill-refs/IMOS/SRS/SST/ghrsst/L3S-1d/ngt/2021/20210725152000-ABOM-L3S_GHRSST-SSTskin-AVHRR_D-1d_night.json'
ds = open_dataset(fn)
display(ds)

<xarray.Dataset>
Dimensions:                          (lat: 4500, lon: 6000, time: 1)
Coordinates:
  * lat                              (lat) float32 19.99 19.97 ... -69.97 -69.99
  * lon                              (lon) float32 70.01 70.03 ... 190.0 190.0
  * time                             (time) datetime64[ns] 2021-07-25T15:20:00
Data variables: (12/13)
    dt_analysis                      (time, lat, lon) float32 dask.array<chunksize=(1, 2250, 3000), meta=np.ndarray>
    l2p_flags                        (time, lat, lon) float32 dask.array<chunksize=(1, 1500, 2000), meta=np.ndarray>
    quality_level                    (time, lat, lon) float32 dask.array<chunksize=(1, 2250, 3000), meta=np.ndarray>
    satellite_zenith_angle           (time, lat, lon) float32 dask.array<chunksize=(1, 2250, 3000), meta=np.ndarray>
    sea_ice_fraction                 (time, lat, lon) float32 dask.array<chunksize=(1, 2250, 3000), meta=np.ndarray>
    sea_ice_fraction_dtime_from_sst  (time, lat, lon) float32 dask.array<chunksize=(1, 2250, 3000), meta=np.ndarray>
    ...                               ...
    sses_bias                        (time, lat, lon) float32 dask.array<chunksize=(1, 2250, 3000), meta=np.ndarray>
    sses_count                       (time, lat, lon) float32 dask.array<chunksize=(1, 2250, 3000), meta=np.ndarray>
    sses_standard_deviation          (time, lat, lon) float32 dask.array<chunksize=(1, 2250, 3000), meta=np.ndarray>
    sst_dtime                        (time, lat, lon) float64 dask.array<chunksize=(1, 1500, 2000), meta=np.ndarray>
    wind_speed                       (time, lat, lon) float32 dask.array<chunksize=(1, 2250, 3000), meta=np.ndarray>
    wind_speed_dtime_from_sst        (time, lat, lon) float32 dask.array<chunksize=(1, 2250, 3000), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                CF-1.6
    Metadata_Conventions:       Unidata Dataset Discovery v1.0
    Metadata_Link:              TBA
    acknowledgment:             Any use of these data requires the following ...
    cdm_data_type:              grid
    comment:                    HRPT AVHRR experimental L3 retrieval produced...
    ...                         ...
    summary:                    Skin SST retrievals produced from stitching t...
    time_coverage_end:          20210725T225520Z
    time_coverage_start:        20210725T111633Z
    title:                      IMOS L3S Nighttime gridded multiple-sensor mu...
    uuid:                       73a8d665-6b25-4f5b-80f8-8d4b296da543
    westernmost_longitude:      70.01000213623047

In [7]:
list(ds.data_vars)

['dt_analysis',
 'l2p_flags',
 'quality_level',
 'satellite_zenith_angle',
 'sea_ice_fraction',
 'sea_ice_fraction_dtime_from_sst',
 'sea_surface_temperature',
 'sses_bias',
 'sses_count',
 'sses_standard_deviation',
 'sst_dtime',
 'wind_speed',
 'wind_speed_dtime_from_sst']

### Find files with different chunking in the netCDF - mostly in 2016

In [8]:
# fs = fsspec.filesystem('s3',profile='default')

In [9]:
# files = fs.glob('s3://imos-data-pixeldrill-refs/IMOS/SRS/SST/ghrsst/L3S-1d/ngt/2016/201605*')
# for f in files:
#     ds = open_dataset(f)
#     print(f + str(ds['satellite_zenith_angle'].encoding['chunks']))

In [10]:
# Badly chunked file: imos-data-pixeldrill/IMOS/SRS/SST/ghrsst/L3S-1d/ngt/2016/20161001152000-ABOM-L3S_GHRSST-SSTskin-AVHRR_D-1d_night.nc
# fs.delete('imos-data-pixeldrill-refs/IMOS/SRS/SST/ghrsst/L3S-1d/ngt/2016/20161001152000-ABOM-L3S_GHRSST-SSTskin-AVHRR_D-1d_night.json')

### Aggregate json at different time scales - deals with different chunk layouts by creating separate stacks for each layout

In [11]:
def aggregate_json(root='s3://imos-data-pixeldrill-refs/IMOS/SRS/SST/ghrsst/L3S-1d/ngt/',mask='2021/202101',dest='./refs_monthly/',suffix='_ABOM-L3S_GHRSST-SSTskin-AVHRR_D-1d_night.json'):
    import os
    
    def _preprocess(ds):
        variables = ['dt_analysis',
                     'l2p_flags',
                     'quality_level',
                     'satellite_zenith_angle',
                     'sea_surface_temperature',
                     'sses_bias',
                     'sses_count',
                     'sses_standard_deviation',
                     'sst_dtime']
        ds = ds[variables]
        return ds
    
    fs = fsspec.filesystem('s3',profile='default')
    references = fs.glob(f"{root}{mask}*.json")
        
    if len(references) >= 1:
        so = dict(
            protocol='s3',
            profile='default', 
            default_fill_cache=False, 
            default_cache_type='first',
        )
        
        #Deal with different chunk sizes - create a separate aggregate file for each chunking layout
        chunking = {}
        for r in references:
            ds=open_dataset(r)
            key = ds['sea_surface_temperature'].chunks
            if key in chunking.keys():
                chunking[key].append(r)
            else:
                chunking[key] = [r,]
        
        #Label each set with a, b, c, ...
        labels = [chr(i) for i in range(97,97+len(chunking.keys()))]
        agg_files=[]
        for i, (chunks, refs) in enumerate(chunking.items()):
            
            #setup output location
            agg_file = f"{dest}{mask}{suffix}".replace('.json',f'_{labels[i]}.json')
            dirname = os.path.dirname(agg_file)
            os.makedirs(dirname,exist_ok=True)
            
            if len(refs) == 1: # Only one refence in this set... just use the source reference file
                fs.get(refs[0],agg_file)
            else: # otherwise join the references into one file
                mzz = MultiZarrToZarr(
                    refs,
                    storage_options=so,
                    remote_protocol="s3",
                    remote_options={'profile': 'default'},
                    xarray_concat_args=dict(dim='time',coords='minimal',join='override',compat='override',combine_attrs='override', fill_value=''),
                    preprocess=_preprocess
                )

                try:
                    mzz.translate(agg_file)
                except NotImplementedError as ex:
                    agg_file = f'ERROR(CHUNK): {agg_file} {str(ex)}'
                except Exception as ex:
                    agg_file = f'ERROR(UNKOWN): {agg_file} {str(ex)}'
                    raise ex
            
            agg_files.append(agg_file)
            
        return {mask: agg_files}
    else:
        return {mask: 'ERROR(NOFILES)'}

In [12]:
# aggregate_json(mask='2016/201610')

### Monthly aggregates

In [13]:
from dask import delayed, compute
d_aggregate_json=delayed(aggregate_json)
futures=[]
for year in range(1992,2022):
    for month in range(1,13):
        futures.append(d_aggregate_json(dest='./refs_monthly/',
                                        mask=f'{year}/{year}{month:02d}'))   
results = compute(futures)[0]

### Yearly aggregates

In [14]:
futures=[]
for year in range(1992,2022):
        futures.append(d_aggregate_json(dest='./refs_yearly/',
                                        mask=f'{year}/{year}'))   
results = compute(futures)[0]

### Upload to S3 bucket

In [15]:
def upload_to_s3(f,urlpath='imos-data-pixeldrill-refs/IMOS/SRS/SST/ghrsst/L3S-1d/ngt/agg_monthly/'):
    import fsspec
    fs = fsspec.filesystem('s3',profile='default')
    fn = os.path.basename(f)
    fs.put_file(f,urlpath + fn)
    return "Success"

In [16]:
#monthly files
agg_files = sorted(glob.glob('./refs_monthly/*/*.json'))
futures = client.map(upload_to_s3,agg_files,urlpath='imos-data-pixeldrill-refs/IMOS/SRS/SST/ghrsst/L3S-1d/ngt/agg_monthly/')
results = client.gather(futures)

In [17]:
#yearly files
agg_files = sorted(glob.glob('./refs_yearly/*/*.json'))
futures = client.map(upload_to_s3,agg_files,urlpath='imos-data-pixeldrill-refs/IMOS/SRS/SST/ghrsst/L3S-1d/ngt/agg_yearly/')
results = client.gather(futures)